In [47]:
import pandas as pd
import gzip
import json

def getDataFrameFromZipFile(filePath):
    data = []
    # Open the .gz file and read its contents
    with gzip.open(filePath, 'rt') as file:
        lines = file.readlines()

    for line in lines:
        try:
            # Convert each line to a dictionary
            data_dict = eval(line)
            data.append(data_dict)
        except json.JSONDecodeError:
            # Skip the line if it's not valid JSON
            continue
    # Create a Pandas DataFrame from the list of dictionaries
    df = pd.DataFrame(data)
    return df

def cleanData(data, dateToProcess='2021-08-20'):
    # Convert timestamp columns to datetime
    data['timestamp'] = pd.to_datetime(data['timestamp'])
    data['last_trade_time'] = pd.to_datetime(data['last_trade_time'])
    
    data.sort_values('timestamp', inplace=True)
    # Drop duplicate timestamps
    data.drop_duplicates(subset='timestamp', inplace=True)
    
    # Handle null data
    data.dropna(subset=['last_price', 'volume', 'oi'], inplace=True)
    
    # Filter data for 20th August 2021
    data = data[data['timestamp'].dt.date == pd.to_datetime(dateToProcess).date()]
    
    # Filter data for 9:15 to 3:30
    data = data[(data['timestamp'].dt.time >= pd.to_datetime('9:15').time()) &
                (data['timestamp'].dt.time <= pd.to_datetime('15:30').time())]
    
    # Filter data for Monday to Friday
    data = data[data['timestamp'].dt.weekday < 5]
    data.set_index('timestamp', inplace=True)
    
    return data

def rollupData(data):
    # Resample data to 1-minute intervals
    resampled_data = data.resample('1T').agg({
        'last_price': 'ohlc',
        'volume': 'sum',
        'oi': 'last'
    })
    # Rename the columns
    resampled_data.columns = ['open', 'high', 'low', 'close', 'volume', 'oi']

    # Drop rows with missing values
    resampled_data.dropna(subset=['open', 'high', 'low', 'close'], inplace=True)

    return resampled_data


def combinePremiumData(call_data, put_data):
    # Combine put and call data
    merged_df = pd.merge(call_data, put_data, on='timestamp', how='inner', suffixes=('_call', '_put'))
    
    # Specify the column names for summation
    columns_to_sum = ['open', 'high', 'low', 'close', 'volume', 'oi']

    # Perform summation for each column
    for col in columns_to_sum: # we can also do fillna(method='bfill')
        merged_df[f'{col}'] = merged_df[f'{col}_call'].fillna(0) + merged_df[f'{col}_put'].fillna(0) 
    # Drop unnecessary columns
    merged_df.drop([f'{col}_call' for col in columns_to_sum] +
                   [f'{col}_put' for col in columns_to_sum], axis=1, inplace=True)
    return merged_df



rawDataCall = getDataFrameFromZipFile('BANKNIFTY21SEP35300CE_2021_08_20.txt.gz')
rawDataPut = getDataFrameFromZipFile('BANKNIFTY21SEP35300PE_2021_08_20.txt.gz')

# Clean call data
call_data = cleanData(rawDataCall)
# Clean put data
put_data = cleanData(rawDataPut)
call_data.head(50)

,tradable,mode,instrument_token,last_price,last_quantity,average_price,volume,buy_quantity,sell_quantity,ohlc,change,last_trade_time,oi,oi_day_high,oi_day_low,depth
timestamp,,,,,,,,,,,,,,,,
2021-08-20 09:15:01,True,full,12612610,1064.30,25,0.00,0,625,0,"{'open': 0.0, 'high': 0.0, 'low': 0.0, 'close'...",0.000000,2021-08-18 15:26:48,500,0,0,"{'buy': [{'quantity': 100, 'price': 501.1, 'or..."
2021-08-20 09:15:03,True,full,12612610,1064.30,25,0.00,0,1875,1650,"{'open': 0.0, 'high': 0.0, 'low': 0.0, 'close'...",0.000000,2021-08-18 15:26:48,500,0,0,"{'buy': [{'quantity': 50, 'price': 507.05, 'or..."
2021-08-20 09:15:04,True,full,12612610,1064.30,25,0.00,0,1675,1625,"{'open': 0.0, 'high': 0.0, 'low': 0.0, 'close'...",0.000000,2021-08-18 15:26:48,500,0,0,"{'buy': [{'quantity': 100, 'price': 744.1, 'or..."
2021-08-20 09:15:18,True,full,12612610,1064.30,25,0.00,0,4750,5125,"{'open': 0.0, 'high': 0.0, 'low': 0.0, 'close'...",0.000000,2021-08-18 15:26:48,500,0,0,"{'buy': [{'quantity': 50, 'price': 752.35, 'or..."
2021-08-20 09:15:19,True,full,12612610,1064.30,25,0.00,0,4750,5600,"{'open': 0.0, 'high': 0.0, 'low': 0.0, 'close'...",0.000000,2021-08-18 15:26:48,500,0,0,"{'buy': [{'quantity': 50, 'price': 753.9, 'ord..."
2021-08-20 09:15:34,True,full,12612610,1064.30,25,0.00,0,5450,4325,"{'open': 0.0, 'high': 0.0, 'low': 0.0, 'close'...",0.000000,2021-08-18 15:26:48,500,0,0,"{'buy': [{'quantity': 50, 'price': 773.15, 'or..."
2021-08-20 09:15:35,True,full,12612610,1064.30,25,0.00,0,4900,5150,"{'open': 0.0, 'high': 0.0, 'low': 0.0, 'close'...",0.000000,2021-08-18 15:26:48,500,0,0,"{'buy': [{'quantity': 50, 'price': 774.25, 'or..."
2021-08-20 09:15:47,True,full,12612610,1064.30,25,0.00,0,5425,4125,"{'open': 0.0, 'high': 0.0, 'low': 0.0, 'close'...",0.000000,2021-08-18 15:26:48,500,0,0,"{'buy': [{'quantity': 50, 'price': 824.9, 'ord..."
2021-08-20 09:15:48,True,full,12612610,1064.30,25,0.00,0,4875,3400,"{'open': 0.0, 'high': 0.0, 'low': 0.0, 'close'...",0.000000,2021-08-18 15:26:48,500,0,0,"{'buy': [{'quantity': 25, 'price': 825.75, 'or..."


In [48]:
# Roll up data to 1-minute level
call_rollup = rollupData(call_data)
put_rollup = rollupData(put_data)

call_rollup

,open,high,low,close,volume,oi
timestamp,,,,,,
2021-08-20 09:15:00,1064.30,1064.30,1064.30,1064.30,0,500.0
2021-08-20 09:16:00,840.20,872.60,832.95,840.00,1725,500.0
2021-08-20 09:17:00,840.00,840.00,818.95,818.95,4400,625.0
2021-08-20 09:18:00,818.95,818.95,818.95,818.95,16500,625.0
2021-08-20 09:19:00,818.95,837.20,818.95,837.20,17025,625.0
...,...,...,...,...,...,...
2021-08-20 15:25:00,769.00,770.85,769.00,770.85,64050,6400.0
2021-08-20 15:26:00,770.85,770.85,770.85,770.85,21400,6400.0
2021-08-20 15:27:00,770.85,770.85,770.85,770.85,64200,6400.0


In [49]:
put_rollup

,open,high,low,close,volume,oi
timestamp,,,,,,
2021-08-20 09:15:00,760.00,974.75,760.00,881.45,2800,3050
2021-08-20 09:16:00,864.05,864.05,864.05,864.05,2675,3050
2021-08-20 09:17:00,864.05,864.05,864.05,864.05,3300,3050
2021-08-20 09:18:00,919.40,919.40,919.40,919.40,2250,3025
2021-08-20 09:19:00,919.40,919.40,919.40,919.40,18200,3025
...,...,...,...,...,...,...
2021-08-20 15:25:00,862.75,862.75,860.40,860.40,47475,4625
2021-08-20 15:26:00,860.90,860.90,860.90,860.90,47675,4625
2021-08-20 15:27:00,860.90,860.90,860.90,860.90,31850,4625


In [50]:
# Combine put and call data
premium_data = combinePremiumData(call_rollup, put_rollup)

# Print the combined premium data
premium_data['year'] = premium_data.index.year
premium_data['date'] = premium_data.index.date
premium_data['month'] = premium_data.index.month
premium_data

,open,high,low,close,volume,oi,year,date,month
timestamp,,,,,,,,,
2021-08-20 09:15:00,1824.30,2039.05,1824.30,1945.75,2800,3550.0,2021,2021-08-20,8
2021-08-20 09:16:00,1704.25,1736.65,1697.00,1704.05,4400,3550.0,2021,2021-08-20,8
2021-08-20 09:17:00,1704.05,1704.05,1683.00,1683.00,7700,3675.0,2021,2021-08-20,8
2021-08-20 09:18:00,1738.35,1738.35,1738.35,1738.35,18750,3650.0,2021,2021-08-20,8
2021-08-20 09:19:00,1738.35,1756.60,1738.35,1756.60,35225,3650.0,2021,2021-08-20,8
...,...,...,...,...,...,...,...,...,...
2021-08-20 15:25:00,1631.75,1633.60,1629.40,1631.25,111525,11025.0,2021,2021-08-20,8
2021-08-20 15:26:00,1631.75,1631.75,1631.75,1631.75,69075,11025.0,2021,2021-08-20,8
2021-08-20 15:27:00,1631.75,1631.75,1631.75,1631.75,96050,11025.0,2021,2021-08-20,8


In [51]:
start_time = pd.Timestamp('2021-08-20 09:15:00')
end_time = pd.Timestamp('2021-08-20 15:29:00')
reference_df = pd.DataFrame(index=pd.date_range(start_time, end_time, freq='1min'))

# Find the missing timestamps
missing_timestamps = reference_df[~reference_df.index.isin(premium_data.index)]

# Print the missing timestamps
missing_timestamps

""
2021-08-20 12:55:00


In [52]:
#Final Function to get the premium data by giving filePath and dateToProcess
def getStraddlePremiumData(callDataFilePath, putDataFilePath, dateToProcess='2021-08-20'):
    callDataRaw = cleanData(getDataFrameFromZipFile(callDataFilePath), dateToProcess)
    putDataRaw = cleanData(getDataFrameFromZipFile(putDataFilePath), dateToProcess)
    callRollUp = rollupData(callDataRaw)
    putRollUp = rollupData(putDataRaw)
    combinedData = combinePremiumData(callRollUp, putRollUp)
    combinedData['year'] = combinedData.index.year
    combinedData['date'] = combinedData.index.date
    combinedData['month'] = combinedData.index.month
    return combinedData

straddleData = getStraddlePremiumData('BANKNIFTY21SEP35300CE_2021_08_20.txt.gz', 'BANKNIFTY21SEP35300PE_2021_08_20.txt.gz', '2021-08-20')
straddleData

,open,high,low,close,volume,oi,year,date,month
timestamp,,,,,,,,,
2021-08-20 09:15:00,1824.30,2039.05,1824.30,1945.75,2800,3550.0,2021,2021-08-20,8
2021-08-20 09:16:00,1704.25,1736.65,1697.00,1704.05,4400,3550.0,2021,2021-08-20,8
2021-08-20 09:17:00,1704.05,1704.05,1683.00,1683.00,7700,3675.0,2021,2021-08-20,8
2021-08-20 09:18:00,1738.35,1738.35,1738.35,1738.35,18750,3650.0,2021,2021-08-20,8
2021-08-20 09:19:00,1738.35,1756.60,1738.35,1756.60,35225,3650.0,2021,2021-08-20,8
...,...,...,...,...,...,...,...,...,...
2021-08-20 15:25:00,1631.75,1633.60,1629.40,1631.25,111525,11025.0,2021,2021-08-20,8
2021-08-20 15:26:00,1631.75,1631.75,1631.75,1631.75,69075,11025.0,2021,2021-08-20,8
2021-08-20 15:27:00,1631.75,1631.75,1631.75,1631.75,96050,11025.0,2021,2021-08-20,8


In [46]:
straddleData.to_csv('straddlePremiumData.csv')